# Week 3 Assignment 4
##### Georgia  - Team 5

#### The dataset, collected by Davis et al in the 1930s, shows a simple 2-node network:  the attendance of 18 Southern Women at 14 social events. The result is a person-by-event matrix: cell (i,j) is 1 if person i attended social event j, and 0 otherwise. http://vlado.fmf.uni-lj.si/pub/networks/data/ucinet/ucidata.htm#davis

#### Goal: Make inferences about the relationships between (1) the women, and (2) the social events. 

##### Import data from url

In [5]:
import urllib2

for line in urllib2.urlopen("http://vlado.fmf.uni-lj.si/pub/networks/data/Ucinet/davis.dat"):
    print line

DL

NR=18 NC=14

FORMAT = FULLMATRIX DIAGONAL PRESENT

ROW LABELS:

EVELYN

LAURA

THERESA

BRENDA

CHARLOTTE

FRANCES

ELEANOR

PEARL

RUTH

VERNE

MYRNA

KATHERINE

SYLVIA

NORA

HELEN

DOROTHY

OLIVIA

FLORA

COLUMN LABELS:

E1

E2

E3

E4

E5

E6

E7

E8

E9

E10

E11

E12

E13

E14

DATA:

 1 1 1 1 1 1 0 1 1 0 0 0 0 0

 1 1 1 0 1 1 1 1 0 0 0 0 0 0

 0 1 1 1 1 1 1 1 1 0 0 0 0 0

 1 0 1 1 1 1 1 1 0 0 0 0 0 0

 0 0 1 1 1 0 1 0 0 0 0 0 0 0

 0 0 1 0 1 1 0 1 0 0 0 0 0 0

 0 0 0 0 1 1 1 1 0 0 0 0 0 0

 0 0 0 0 0 1 0 1 1 0 0 0 0 0

 0 0 0 0 1 0 1 1 1 0 0 0 0 0

 0 0 0 0 0 0 1 1 1 0 0 1 0 0

 0 0 0 0 0 0 0 1 1 1 0 1 0 0

 0 0 0 0 0 0 0 1 1 1 0 1 1 1

 0 0 0 0 0 0 1 1 1 1 0 1 1 1

 0 0 0 0 0 1 1 0 1 1 1 1 1 1

 0 0 0 0 0 0 1 1 0 1 1 1 1 1

 0 0 0 0 0 0 0 1 1 1 0 1 0 0

 0 0 0 0 0 0 0 0 1 0 1 0 0 0

 0 0 0 0 0 0 0 0 1 0 1 0 0 0



# Python code to create dataset

In [4]:
#!/usr/bin/env python
"""
Davis Southern Club Women

Shows how to make unipartite projections of the graph and compute the
properties of those graphs.

These data were collected by Davis et al. in the 1930s.
They represent observed attendance at 14 social events by 18 Southern women.
The graph is bipartite (clubs, women).
"""
import networkx as nx
import networkx.algorithms.bipartite as bipartite

G = nx.davis_southern_women_graph()
women = G.graph['top']
clubs = G.graph['bottom']

print("Biadjacency matrix")
print(bipartite.biadjacency_matrix(G,women,clubs))

# project bipartite graph onto women nodes
W = bipartite.projected_graph(G, women)
print('')
print("#Friends, Member")
for w in women:
    print('%d %s' % (W.degree(w),w))

# project bipartite graph onto women nodes keeping number of co-occurence
# the degree computed is weighted and counts the total number of shared contacts
W = bipartite.weighted_projected_graph(G, women)
print('')
print("#Friend meetings, Member")
for w in women:
    print('%d %s' % (W.degree(w,weight='weight'),w))

KeyError: 'top'